In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !cp drive/"MyDrive"/train.pkl train.pkl
# !cp drive/"MyDrive"/test_no_target.pkl test_no_target.pkl  

In [3]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import random
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

In [4]:
device = torch.device("cuda")
torch.cuda.empty_cache()
device

device(type='cuda')

In [5]:
np.random.seed(2137)
torch.manual_seed(2137)
random.seed(2137)

In [6]:
train_raw  = pd.read_pickle("./train.pkl")
classes_count = torch.zeros(5)
for x in range(len(train_raw)):
    classes_count[train_raw[x][1]] += 1;
classes_count


tensor([1630.,  478.,  154.,  441.,  236.])

In [7]:
#subsequencing
sequence_limit = 400
train_cropped = []
for x in range(len(train_raw)):
#     parts = int(len(train_raw[x][0])/sequence_limit)+1
#     for i in range(parts):
#         train_cropped.append((train_raw[x][0][i*sequence_limit:min((i+1)*sequence_limit, len(train_raw[x][0]))], train_raw[x][1]))
    train_cropped.append((train_raw[x][0][:min(sequence_limit, len(train_raw[x][0]))], train_raw[x][1]))

In [8]:
train_raw  =train_cropped
classes_count = torch.zeros(5)
for x in range(len(train_raw)):
      classes_count[train_raw[x][1]] += 1
classes_weight = torch.ones(5)*torch.max(classes_count)/classes_count
classes_weight


tensor([ 1.0000,  3.4100, 10.5844,  3.6961,  6.9068])

In [9]:
#downsampling
class_limit = 600
classes_count = torch.zeros(5)
train_seq = []
train_label = []
for x in range(len(train_raw)):
#   if classes_count[train_raw[x][1]] < class_limit:
    if True:
        classes_count[train_raw[x][1]] += 1
        train_seq.append(torch.from_numpy(train_raw[x][0]))
        train_label.append(train_raw[x][1])
torch.sum(classes_count)

tensor(2939.)

In [10]:
#padding
train_seq = pad_sequence(train_seq, batch_first=False, padding_value=0)
train_seq.size()

torch.Size([400, 2939])

In [11]:
data = torch.transpose(train_seq.float(),0, 1)
data_targets = torch.Tensor(train_label).long()
train_indices = np.random.rand(len(data))>0.3
test_indices = ~train_indices
train_set = torch.utils.data.TensorDataset(data[train_indices], data_targets[train_indices])
train_loader = DataLoader(train_set, batch_size=8, shuffle=True)
test_data, test_targets = data[test_indices], data_targets[test_indices]
train_loader

In [12]:
class MusicianFinder(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, out_size):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size=input_size, hidden_size= hidden_size, num_layers = num_layers)
        self.inc0 = nn.Conv1d(1, 1, (1, 11), stride=3)
        self.inc1 = nn.Conv1d(1, 1, (1, 33), stride=7)
        self.inc2 = nn.Conv1d(1, 1, (1, 55), stride=15)
        self.inc3 = nn.Conv1d(1, 1, (1, 77), stride=35)
        self.mpinc0 = nn.MaxPool1d(5)
        self.mpinc1 = nn.MaxPool1d(5)
        self.mpinc2 = nn.MaxPool1d(5)
        self.mpinc3 = nn.MaxPool1d(5)
        self.lin0 = nn.Linear(sequence_limit, 50)
        self.d0 = nn.Dropout(0.6)
        self.act0 =  nn.Sigmoid()
        self.lin1 = nn.Linear(hidden_size*sequence_limit, 150)
        self.d1 = nn.Dropout(0.6)
        self.act1 =  nn.ReLU()
        self.lin2 = nn.Linear(hidden_size*sequence_limit+7606, 500)
        self.d2 = nn.Dropout(0.6)
        self.act2 =  nn.Sigmoid()
        self.lin3 = nn.Linear(500, out_size)

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        state = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return hidden, state

    def forward(self, x, hidden):
#         emb = self.lin0(torch.squeeze(x, 2))
#         emb = self.d0(emb)
#         emb = self.act0(emb) 
        
        x = torch.transpose(x,0,1)
        all_outputs, hidden = self.lstm(x, hidden)
        all_outputs = torch.transpose(all_outputs,0,1)
        out = torch.flatten(all_outputs, 1)
        
        inc = out.unsqueeze(1).unsqueeze(2)
        linc0 = self.inc1(inc)
        linc1 = self.inc1(inc)
        linc2 = self.inc2(inc)
        linc3 = self.inc3(inc)
        linc0, linc1, linc2, linc3 = linc0.squeeze(2), linc1.squeeze(2), linc2.squeeze(2), linc3.squeeze(2)
#         linc0 = self.mpinc0(linc0)
#         linc1 = self.mpinc1(linc1)
#         linc2 = self.mpinc2(linc2)
#         linc3 = self.mpinc3(linc3)
        linc0, linc1, linc2, linc3 = linc0.squeeze(1), linc1.squeeze(1), linc2.squeeze(1), linc3.squeeze(1)
        inc = torch.cat((linc0,linc1,linc2,linc3),1)
        
#         x = self.lin1(out)
#         x = self.d1(x)
#         x = self.act1(x)
        x = torch.cat((out, inc),1)
        x = self.lin2(x)
        x = self.d2(x)
        x = self.act2(x)
        x = self.lin3(x)
        return x, hidden

model = MusicianFinder(1,50,2,5).to(device)

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001, weight_decay = 0.0001)
loss_fun = nn.CrossEntropyLoss(weight = classes_weight.to(device) )
for epoch in range(100):
    epoch_loss = 0
    for x, targets in train_loader:
        x = x.to(device).unsqueeze(2)
        targets = targets.to(device)
        hidden, state = model.init_hidden(x.size(0))
        hidden, state = hidden.to(device), state.to(device)
        preds, _ = model(x, (hidden, state))
        preds = preds.squeeze(1)
        optimizer.zero_grad()
        loss = loss_fun(preds, targets)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
    if epoch % 2 == 0:
        print(f"Epoch: {epoch}, loss: {epoch_loss:.3}")
    if epoch % 10 == 0:
        with torch.no_grad():
            hidden, state = model.init_hidden(len(test_data))
            hidden, state = hidden.to(device), state.to(device)
            preds, _ = model(test_data.to(device).unsqueeze(2), (hidden, state))
        print(f"Accuracy: {(torch.argmax(preds,1).cpu() == test_targets).sum().item()/len(test_targets)}")


Epoch: 0, loss: 4.12e+02
Accuracy: 0.2929061784897025
Epoch: 2, loss: 3.85e+02
Epoch: 4, loss: 3.45e+02
Epoch: 6, loss: 3.31e+02
Epoch: 8, loss: 3.08e+02
Epoch: 10, loss: 2.88e+02
Accuracy: 0.5835240274599542
Epoch: 12, loss: 2.62e+02
Epoch: 14, loss: 2.19e+02
Epoch: 16, loss: 1.74e+02
Epoch: 18, loss: 1.36e+02
Epoch: 20, loss: 1.05e+02
Accuracy: 0.6567505720823799
Epoch: 22, loss: 77.1
Epoch: 24, loss: 54.7
Epoch: 26, loss: 42.9
Epoch: 28, loss: 31.6
Epoch: 30, loss: 26.8
Accuracy: 0.6739130434782609
Epoch: 32, loss: 27.2
Epoch: 34, loss: 30.6
Epoch: 36, loss: 19.0
Epoch: 38, loss: 17.2
Epoch: 40, loss: 12.0
Accuracy: 0.6853546910755148
Epoch: 42, loss: 39.2
Epoch: 44, loss: 19.4
Epoch: 46, loss: 9.23
Epoch: 48, loss: 4.63
Epoch: 50, loss: 6.89
Accuracy: 0.6498855835240275
Epoch: 52, loss: 51.9
Epoch: 54, loss: 33.4
Epoch: 56, loss: 10.8
Epoch: 58, loss: 4.27
Epoch: 60, loss: 2.76
Accuracy: 0.6750572082379863
Epoch: 62, loss: 1.89
Epoch: 64, loss: 17.1
Epoch: 66, loss: 42.7
Epoch: 68,

In [14]:
classes_count = torch.zeros(5)
for x in range(len(test_targets)):
      classes_count[test_targets[x]] += 1;
classes_count

tensor([481., 153.,  38., 136.,  66.])

In [15]:
with torch.no_grad():
    hidden, state = model.init_hidden(len(test_data))
    hidden, state = hidden.to(device), state.to(device)
    preds, _ = model(test_data.to(device).unsqueeze(2), (hidden, state))
print(f"Accuracy: {(torch.argmax(preds,1).cpu() == test_targets).sum().item()/len(test_targets)}")
# for x in range(len(preds)):
#     print(f"{torch.argmax(preds[x])} =? {test_targets[x]}")
torch.argmax(preds,1).cpu()

Accuracy: 0.6739130434782609


tensor([0, 0, 0, 0, 0, 3, 3, 1, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 1, 0, 0, 1, 1,
        0, 3, 3, 4, 0, 0, 0, 0, 3, 3, 1, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0,
        3, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 2,
        0, 0, 3, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3, 0, 1, 0, 0, 3, 0, 0, 0,
        0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 0, 3, 0, 0,
        0, 0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 2, 3, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 2, 4, 0, 0, 0, 3, 0, 0, 0, 0,

In [16]:
def save_tens(t):
    f = open("BartoszRomanowska.csv", "w")
    for i in t:
        f.write(f"{i}\n")
    f.close()

In [17]:
test_raw  = pd.read_pickle("./test_no_target.pkl")
len(test_raw)

1103

In [18]:
test_seq = []
for x in range(len(test_raw)):
    test_seq.append(torch.from_numpy(test_raw[x][:min(sequence_limit, len(test_raw[x]))]))
test_seq = pad_sequence(test_seq, batch_first=False, padding_value=0)
test_seq = torch.transpose(test_seq.float(),0,1)

In [19]:
with torch.no_grad():
    hidden, state = model.init_hidden(len(test_seq))
    hidden, state = hidden.to(device), state.to(device)
    preds, _ = model(test_seq.to(device).unsqueeze(2), (hidden, state))
    save_tens(torch.argmax(preds,1))

RuntimeError: CUDA out of memory. Tried to allocate 926.00 MiB (GPU 0; 2.00 GiB total capacity; 400.76 MiB already allocated; 0 bytes free; 986.00 MiB reserved in total by PyTorch)

In [ ]:
hmm = torch.Tensor([1,2,3,4,5,6,7,8,9,10])
# for i in range(10):
hmm = torch.stack((hmm,hmm+1),0)
for i in range(3):
    hmm = torch.cat((hmm,4*hmm),0)
hmm = hmm.unsqueeze(1).unsqueeze(2)
print(hmm.size())
m = nn.Conv1d(1, 1, (1,3), stride=1)
hmm = m(hmm)
print(hmm.size())
hmm = hmm.squeeze(2)
m = nn.MaxPool1d(3, stride=2)
hmm = m(hmm)
hmm = hmm.squeeze(1)
hmm



In [ ]:
>>> m = nn.Conv1d(16, 33, 3, stride=1)
>>> input = torch.randn(20, 16, 50)
>>> output = m(input)

In [ ]:
output.size()